# Quick Start Guide

## Introduction

This quick start guide is designed to provide you with a concise overview of GAMSPy and its key features. By the end of this guide, you'll have a solid understanding of how to create basic mathematical models using GAMSPy. For more advanced features, we recommend exploring our comprehensive [user guide](https://gamspy.readthedocs.io/en/latest/user/index.html) and the extensive [model library](https://github.com/GAMS-dev/gamspy-examples).

While not mandatory, having a basic understanding of Python programming and familiarity with the [Pandas library](https://pandas.pydata.org/) will be helpful for following this tutorial.

## A Transportation Problem

In this guide, we'll delve into an example of the transportation problem. This classic scenario involves managing supplies from various plants to meet demands at multiple markets for a single commodity. Additionally, we have the unit costs associated with shipping the commodity from plants to markets. The fundamental economic question here is: 

> How can we optimize the shipment quantities between each plant and market to minimize the total transport cost?

The problem's algebraic representation typically takes the following format:

Indices (Sets):

- $i$ = plants
- $j$ = markets

Given Data (Parameters):

- $a_i$ = supply of commodity at plant $i$ (in cases)
- $b_j$ = demand for commodity at market $j$ (in cases)
- $c_{ij}$ = cost per unit of shipment between plant $i$ and market $j$

Decision Variables:

- $x_{ij}$ = amount of commodity to ship from plant $i$ to market $j$ where $x_{ij} \ge 0$ for all $i,j$

Constraints:

- Observe supply limit at plant $i$: $\sum_j x_{ij} \le a_i \: \forall i$
- Satisfy demand at market $j$: $\sum_i x_{ij} \ge b_j \: \forall j$
- Objective Function: Minimize $\sum_i \sum_j c_{ij} \cdot x_{ij}$


## Symbol Declaration

In line with our systematic breakdown of the transportation problem into sets, parameters, variables, and constraints, we will adopt a similar approach to define the problem as a GAMSPy `Model`. To do so, it is essential to import the `gamspy` library initially.

In [1]:
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense

### Container

Before we proceed further, let's create a `Container` to encapsulate all the relevant information for our GAMSPy ``Model``. This ``Container`` acts as a centralized hub, gathering essential data, sets, parameters, variables, and constraints, providing a clear structure for our optimization problem.

In [2]:
m = Container()

### Sets

Sets serve as the fundamental building blocks of a GAMSPy ``Model``, directly corresponding to the indices in the algebraic representations of models. In our transportation problem context, we have defined the following indices:

- $i$ = plants
- $j$ = markets

For detailed guidance on using sets, please refer to the [set section](https://gamspy.readthedocs.io/en/latest/user/basics/set.html) of our user guide.

In [3]:
i = Set(container=m, name="i", description="plants")
j = Set(container=m, name="j", description="markets")

The effect of using the above `Set` statements is that we declared two sets, namely $i$ and $j$. Additionally, we provided descriptions to elaborate on their meaning, enhancing the readability of our ``Model``. 

### Parameters

Parameters are declared using the `Parameter` class. Each parameter is assigned a name and a description.  Note that parameter $a_i$ is indexed by $i$. To accommodate these indices, we include the `domain` attribute, pointing to the corresponding set.

In [4]:
a = Parameter(
    container=m, 
    name="a",
    domain=i,
    description="supply of commodity at plant i (in cases)"
)
b = Parameter(
    container=m,
    name="b",
    domain=j,
    description="demand for commodity at market j (in cases)"
)
c = Parameter(
    container=m,
    name="c",
    domain=[i, j],
    description="cost per unit of shipment between plant i and market j",
)

### Variables

GAMSPy variables are declared using `Variable`. Each ``Variable`` is assigned a name, a domain if necessary, a type, and, optionally, a description.

In [5]:
x = Variable(
    container=m,
    name="x",
    domain=[i, j],
    type="Positive",
    description="amount of commodity to ship from plant i to market j",
)

This statement results in the declaration of a shipment variable for each (i,j) pair.

More information on variables can be found in the [variable section](https://gamspy.readthedocs.io/en/latest/user/basics/variable.html) of our user guide.

## Equations
A GAMSPy ``Equation`` can be declared and defined in two separate statements (see [here](https://gamspy.readthedocs.io/en/latest/user/basics/equation.html#combining-equation-declaration-and-definition) how to do this in one statement). The format of the declaration is the same as for other GAMSPy symbols. First comes the keyword, `Equation` in this case, followed by the name, domain and text. The transportation problem has two constraints:

Supply: observe supply limit at plant $i$: $\sum_j x_{ij} \le a_i \: \forall i$

Demand: satisfy demand at market $j$: $\sum_i x_{ij} \ge b_j \: \forall j$

In [6]:
supply = Equation(
    container=m, name="supply", domain=i, description="observe supply limit at plant i"
)
demand = Equation(
    container=m, name="demand", domain=j, description="satisfy demand at market j"
)

The components of an ``Equation`` definition are:
1. The Python variable of the ``Equation`` being defined
2. The domain (optional)
3. Domain restricting conditions (optional)
4. A `=` sign
5. Left hand side expression
6. Relational operator (`==`, `<=`, `>=`)
7. The right hand side expression.

The ``Equation`` definition for the supply constraint of the transportation problem is implemented as follows:

In [7]:
supply[i] = Sum(j, x[i, j]) <= a[i]

Using the same logic as above, we can define the demand equation as follows:

In [8]:
demand[j] = Sum(i, x[i, j]) >= b[j]

More information on equations is given in the [equation section](https://gamspy.readthedocs.io/en/latest/user/basics/equation.html) of our user guide.

## Objective
The objective function of a GAMSPy ``Model`` does not require a separate ``Equation`` declaration. You can either assign the objective expression to a Python variable or use it directly in the ``Model()`` statement of the [next section](#model). An alternative way of declaring the objective with a seperate Equation would look like the following:


In [9]:
obj = Sum((i, j), c[i, j] * x[i, j])

## Model

A GAMSPy `Model()` consolidates constraints, an objective function, a sense (minimize, maximize, and feasibility), and a problem type. It also possesses a name and is associated with a ``Container``.

To define our transportation problem as a GAMSPy ``Model``, we assign it to a Python variable, link it to our ``Container`` (populated with symbols and data), name it "transport", specify the equations, set the problem type as linear program (LP), specify the sense of the objective function (``Sense.MIN``), and point to the objective expression.

GAMSPy allows two alternatives to assign equations to a `Model`:
1. Using a list of equations,
2. Retrieving _all_ equations by calling `m.getEquations()`.

### Using a List of Equations
Using a list of equations is especially useful if you want to define multiple GAMSPy ``Model``s with a subset of the  equations in your ``Container``. For the transportation problem this can be done as follows:

In [10]:
transport = Model(
    m,
    name="transport",
    equations=[supply, demand],
    problem="LP",
    sense=Sense.MIN,
    objective=obj,
)

### Retrieving all Equations
Using `m.getEquations()` is especially convenient if you want to include all equations of your ``Container`` to be associated with your model. For the transportation problem this can be done as follows:

In [11]:
transport = Model(
    m,
    name="transport",
    equations=m.getEquations(),
    problem="LP",
    sense=Sense.MIN,
    objective=obj,
)

More information on the usage of a GAMSPy `Model` can be found in the [model section](https://gamspy.readthedocs.io/en/latest/user/basics/model.html) of our user guide.

## Data

Before solving our model, let's set the data of the model using either Python primitives (i.e. list, tuple etc.) or the [Pandas library](https://pandas.pydata.org/). We'll begin by organizing the necessary information, which we will subsequently feed into our optimization model. There are two ways to declare symbols:

1. Separate declaration and data assignment
2. Combine declaration and data assignment

#### Separate declaration and data assignment

In [12]:
i.setRecords(['seattle', 'san-diego'])
j.setRecords(['new-york', 'chicago', 'topeka'])
a.setRecords([("seattle", 350), ("san-diego", 600)])
b.setRecords([("new-york", 325), ("chicago", 300), ("topeka", 275)])
i.records

,uni,element_text
0,seattle,
1,san-diego,


We assigned members to the sets, establishing a clear connection between the abstract sets and their real-world counterparts.

$i$ = {Seattle, San Diego}

$j$ = {New York, Chicago, Topeka}

To verify the content of a set, you can use `<set name>.records`.

#### Combine declaration and data assignment

In [13]:
import pandas as pd

distances = pd.DataFrame(
    [
        ["seattle", "new-york", 2.5],
        ["seattle", "chicago", 1.7],
        ["seattle", "topeka", 1.8],
        ["san-diego", "new-york", 2.5],
        ["san-diego", "chicago", 1.8],
        ["san-diego", "topeka", 1.4],
    ],
    columns=["from", "to", "distance"]
).set_index(["from", "to"])
d = Parameter(
    container=m,
    name="d",
    domain=[i, j],
    description="distance between plant i and market j",
    records=distances.reset_index(),
)
d.records

,i,j,value
0,seattle,new-york,2.5
1,seattle,chicago,1.7
2,seattle,topeka,1.8
3,san-diego,new-york,2.5
4,san-diego,chicago,1.8
5,san-diego,topeka,1.4


The cost per unit of shipment between plant $i$ and market $j$ is derived from the distance between $i$ and $j$ and can be calculated as follows:

$c_{ij} = \frac{90 \cdot d_{ij}}{1000}$,

where $d_{ij}$ denotes the distance between $i$ and $j$.

We have two options to calculate $c_{ij}$ and assign the data to the GAMSPy parameter:

1. Python assignment - calculation in Python, e.g., using Pandas and `<parameter name>.setRecords()`
2. GAMSPy assignment - calculation in GAMSPy

#### Python Assignment

In [14]:
freight_cost = 90
cost = freight_cost * distances / 1000
c.setRecords(cost.reset_index())
c.records

,i,j,value
0,seattle,new-york,0.225
1,seattle,chicago,0.153
2,seattle,topeka,0.162
3,san-diego,new-york,0.225
4,san-diego,chicago,0.162
5,san-diego,topeka,0.126


#### GAMSPy Assignment

In [15]:
c[i, j] = freight_cost * d[i, j] / 1000
c.records

,i,j,value
0,seattle,new-york,0.225
1,seattle,chicago,0.153
2,seattle,topeka,0.162
3,san-diego,new-york,0.225
4,san-diego,chicago,0.162
5,san-diego,topeka,0.126


## Solve

Once the GAMSPy ``Model`` is defined, it's ready to be solved. The ``solve()`` statement triggers the generation of the specific model instance, creates suitable data structures for the solver, and invokes the solver. To view solver output in the console, the ``sys`` library can be used, passing the ``output=sys.stdout`` attribute to ``transport.solve()``.

In [16]:
import sys

transport.solve(output=sys.stdout)

--- Job _cc823548-24d8-4f42-81e4-82251d6b08b0.gms Start 08/22/24 14:37:27 47.4.1 4b675771 LEX-LEG x86 64bit/Linux
--- Applying:
    /home/muhammet/anaconda3/envs/py38/lib/python3.8/site-packages/gamspy_base/gmsprmun.txt
--- GAMS Parameters defined
    Input /tmp/tmpegp51eu5/_cc823548-24d8-4f42-81e4-82251d6b08b0.gms
    Output /tmp/tmpegp51eu5/_cc823548-24d8-4f42-81e4-82251d6b08b0.lst
    ScrDir /tmp/tmpegp51eu5/tmpggnrlvct/
    SysDir /home/muhammet/anaconda3/envs/py38/lib/python3.8/site-packages/gamspy_base/
    LogOption 3
    Trace /tmp/tmpegp51eu5/_cc823548-24d8-4f42-81e4-82251d6b08b0.txt
    License /home/muhammet/anaconda3/envs/py38/lib/python3.8/site-packages/gamspy_base/user_license.txt
    OptDir /tmp/tmpegp51eu5/
    LimRow 0
    LimCol 0
    TraceOpt 3
    GDX /tmp/tmpegp51eu5/_cc823548-24d8-4f42-81e4-82251d6b08b0out.gdx
    SolPrint 0
    PreviousWork 1
    gdxSymbols newOrChanged
Licensee: GAMSPy Incremental Professional                G240510+0003Cc-GEN
          GAMS    

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,153.675,6,7,LP,CPLEX,0


## Retrieving Results
### Variable Values
The values of the variables in the solution can be retrieved using ``<variable name>.records``. The level specifies the shipment quantities $x_{ij}$. Other variable attributes are the marginal values, lower and upper bounds, and the variable's scaling factor.

In [17]:
x.records

,i,j,level,marginal,lower,upper,scale
0,seattle,new-york,50.0,0.000,0.0,inf,1.0
1,seattle,chicago,300.0,0.000,0.0,inf,1.0
2,seattle,topeka,0.0,0.036,0.0,inf,1.0
3,san-diego,new-york,275.0,0.000,0.0,inf,1.0
4,san-diego,chicago,0.0,0.009,0.0,inf,1.0
5,san-diego,topeka,275.0,0.000,0.0,inf,1.0


### Objective Value
The optimal objective function value can be accessed by `<model name>.objective_value`.

In [18]:
transport.objective_value

153.675